# Importazioni

In [ ]:
# Importazione delle librerie necessarie
import os
import json
import mysql.connector
from dotenv import load_dotenv
from googleapiclient.discovery import build
from google.oauth2 import service_account
from datetime import datetime

# Carica credenziali

In [ ]:
# Carica le variabili d'ambiente dal file .env
load_dotenv()

# Legge le credenziali Google dal file .env
credenziali_json = os.getenv("GOOGLE_CREDENTIALS")
credenziali_dict = json.loads(credenziali_json)

# Corregge i caratteri di a capo nella chiave privata
credenziali_dict['private_key'] = credenziali_dict['private_key'].replace('\\n', '\n')

# Definisce i permessi di accesso (sola lettura)
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

# Crea le credenziali per autenticarsi con Google
credenziali = service_account.Credentials.from_service_account_info(
    credenziali_dict,
    scopes=SCOPES
)

print("✅ Credenziali caricate con successo!")

# Leggi Google Sheet

In [ ]:
# ID del Google Sheet e nome del foglio
ID_FOGLIO  = '14XJF4TtNgnHv_uuJeUcWAjipOYs5-hPYbslC4xXyqwQ'
INTERVALLO = 'Risposte del modulo 1!A:AM'  # ✅ Cambiato da A:AJ a A:AM

# Connessione al servizio Google Sheets
servizio = build('sheets', 'v4', credentials=credenziali)
foglio   = servizio.spreadsheets()

print("📡 Connessione a Google Sheets in corso...")

# Legge i dati dal foglio
risultato = foglio.values().get(
    spreadsheetId=ID_FOGLIO,
    range=INTERVALLO
).execute()

righe = risultato.get('values', [])

if not righe:
    print("❌ Nessun dato trovato nel foglio!")
else:
    print(f"✅ Trovate {len(righe) - 1} risposte nel foglio")
    print(f"📋 Colonne: {righe[0]}")
    print(f"🔢 Numero totale colonne: {len(righe[0])}")  # ✅ Aggiunto per debug

# Connessione MySQL

In [ ]:
# Connessione al database MySQL
conn = mysql.connector.connect(
    host=os.getenv("DB_HOST"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    database=os.getenv("DB_NAME"),
    auth_plugin='mysql_native_password'
)
cursore = conn.cursor()

print("✅ Connesso al database MySQL!")

 # Inserimento dati

In [ ]:
# verificazione la colonna area geografica se insirisce i dati giusti
# DEBUG - Verifica la colonna area_geo
print("🔍 Debug delle prime 5 righe - posizione area_geo:")
for i, riga in enumerate(righe[1:6], start=2):
    riga_completa = riga + [None] * (39 - len(riga))
    print(f"Riga {i}:")
    print(f"  - Lunghezza riga originale: {len(riga)}")
    print(f"  - area_geo (posizione 36): '{riga_completa[36]}'")
    print(f"  - email (posizione 37): '{riga_completa[37]}'")
    print()

In [ ]:
# Connessione al database MySQL (nella stessa cella dell'inserimento)
conn = mysql.connector.connect(
    host=os.getenv("DB_HOST"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    database=os.getenv("DB_NAME"),
    auth_plugin='mysql_native_password'
)
cursore = conn.cursor()

print("✅ Connesso al database MySQL!")

# Query di inserimento (INSERT IGNORE evita i duplicati)
query_inserimento = """
INSERT IGNORE INTO raw_data (
    timestamp, email, contesto, età, genere, area_geo, anni_esperienza,
    Item_1,  Item_2,  Item_3,  Item_4,  Item_5,  Item_6,  Item_7,
    Item_8,  Item_9,  Item_10, Item_11, Item_12, Item_13, Item_14,
    Item_15, Item_16, Item_17, Item_18, Item_19, Item_20, Item_21,
    Item_22, Item_23, Item_24, Item_25, Item_26, Item_27, Item_28,
    Item_29, Item_30
) VALUES (
    %s, %s, %s, %s, %s, %s, %s,
    %s, %s, %s, %s, %s, %s, %s,
    %s, %s, %s, %s, %s, %s, %s,
    %s, %s, %s, %s, %s, %s, %s,
    %s, %s, %s, %s, %s, %s, %s,
    %s, %s
)
"""

# Contatori per il riepilogo finale
inserite = 0
saltate  = 0

# Ciclo su tutte le righe (salta la prima riga che è l'intestazione)
for i, riga in enumerate(righe[1:], start=2):
    try:
        # Aggiunge None se la riga ha meno colonne del previsto
        riga_completa = riga + [None] * (39 - len(riga))

        # Converte il timestamp dal formato Google Sheets
        try:
            ts = datetime.strptime(riga_completa[0], '%d/%m/%Y %H:%M:%S')
        except:
            # Prova formato alternativo
            try:
                ts = datetime.strptime(riga_completa[0], '%d/%m/%Y %H.%M.%S')
            except:
                ts = None

        # Prepara i dati da inserire
        dati = (
            ts,                   # timestamp
            riga_completa[37],    # email (dalla colonna 37 del CSV)
            riga_completa[2],     # contesto
            riga_completa[3],     # età
            riga_completa[4],     # genere
            riga_completa[36],    # area_geo (dalla colonna 36 del CSV)
            riga_completa[5],     # anni_esperienza
            riga_completa[6],     # Item_1
            riga_completa[7],     # Item_2
            riga_completa[8],     # Item_3
            riga_completa[9],     # Item_4
            riga_completa[10],    # Item_5
            riga_completa[11],    # Item_6
            riga_completa[12],    # Item_7
            riga_completa[13],    # Item_8
            riga_completa[14],    # Item_9
            riga_completa[15],    # Item_10
            riga_completa[16],    # Item_11
            riga_completa[17],    # Item_12
            riga_completa[18],    # Item_13
            riga_completa[19],    # Item_14
            riga_completa[20],    # Item_15
            riga_completa[21],    # Item_16
            riga_completa[22],    # Item_17
            riga_completa[23],    # Item_18
            riga_completa[24],    # Item_19
            riga_completa[25],    # Item_20
            riga_completa[26],    # Item_21
            riga_completa[27],    # Item_22
            riga_completa[28],    # Item_23
            riga_completa[29],    # Item_24
            riga_completa[30],    # Item_25
            riga_completa[31],    # Item_26
            riga_completa[32],    # Item_27
            riga_completa[33],    # Item_28
            riga_completa[34],    # Item_29
            riga_completa[35],    # Item_30 (testo libero)
        )

        cursore.execute(query_inserimento, dati)
        inserite += 1

    except Exception as e:
        print(f"⚠️ Riga {i} saltata: {e}")
        saltate += 1

# Salva le modifiche nel database
conn.commit()
print(f"\n✅ Inserite {inserite} nuove righe in raw_data")
print(f"⚠️ Saltate {saltate} righe duplicate")

# Chiude la connessione
cursore.close()
conn.close()